In [1]:
import numpy as np
import scipy

In [46]:
line = np.arange(3).astype(np.int16)
c = np.ones_like(line)

In [49]:
def poly_exp_1D(f, c, sigma):
    """
    Calculates the local polynomial expansion of a 1D signal.
    
    $f ~ x^T A x + B^T x + C$
    
    If f[i] and c[i] are the signal value and certainty of pixel i then
    A[i] is a 1x1 array representing the quadratic term of the polynomial, B[i]
    is a 1-element array representing the linear term, and C[i] is a scalar
    representing the constant term.
    
    Parameters
    ----------
    f
        Input signal
    c
        Certainty of signal
    sigma
        Standard deviation of applicability Gaussian kernel
    Returns
    -------
    A
        Quadratic term of polynomial expansion
    B
        Linear term of polynomial expansion
    C
        Constant term of polynomial expansion
    """
    print("f", f)
    print("c", c)
    # Kernel applicability
    n = int(4 * sigma + 1)
    x = np.arange(-n, n + 1, dtype=np.int32)
    a = np.exp(-(x**2) / (2 * sigma**2))

    # b: calculate b from the paper.
    b = np.stack([np.ones(a.shape), x, x**2], axis=-1)
    print(b.shape)

    # Pre-calculate product of certainty and signal
    cf = c * f

    # G and v are used to calculate "r" from the paper: v = G*r
    # r is the parametrization of the 2nd order polynomial for f
    G = np.empty(list(f.shape) + [b.shape[-1]] * 2)
    v = np.empty(list(f.shape) + [b.shape[-1]])
    #print("G.shape =", G.shape)
    #print("v.shape =", v.shape)

    # Apply cross-correlation

    # Pre-calculate quantities recommended in paper
    ab = np.einsum("i,ij->ij", a, b)
    abb = np.einsum("ij,ik->ijk", ab, b)
    #print("a", a)
    #print("b", b)
    #print("ab", ab)
    #print("abb", abb)

    # Calculate G and v for each pixel with cross-correlation
    print("b.shape[-1]", b.shape[-1])
    for j in range(b.shape[-1]):
        print("weights", abb[..., j])
        print("c", c)
        print("abb[..., j][0]", abb[..., j][0])
        print("G[..., j]", G[..., j])
        G[..., j] = scipy.ndimage.correlate1d(
            c, abb[..., j][0], axis=0, mode="constant", cval=0
        )

    v = scipy.ndimage.correlate1d(
        cf, ab, axis=0, mode="constant", cval=0
    )

    # Solve r for each pixel
    r = np.linalg.solve(G, v)

    # Quadratic term
    A = np.empty(list(f.shape))
    A[..., 0] = r[..., 2]

    # Linear term
    B = np.empty(list(f.shape))
    B[..., 0] = r[..., 1]

    # constant term
    C = r[..., 0]

    # b: [n, n, 6]
    # r: [f, f, 6]
    # f: [f, f]
    # e = b*r - f

    return A, B, C

In [50]:
poly_exp_1D(line, c, sigma=1.0)

f [0 1 2]
c [1 1 1]
(11, 3)
b.shape[-1] 3
weights [[ 3.72665317e-06 -1.86332659e-05  9.31663293e-05]
 [ 3.35462628e-04 -1.34185051e-03  5.36740205e-03]
 [ 1.11089965e-02 -3.33269896e-02  9.99809688e-02]
 [ 1.35335283e-01 -2.70670566e-01  5.41341133e-01]
 [ 6.06530660e-01 -6.06530660e-01  6.06530660e-01]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.06530660e-01  6.06530660e-01  6.06530660e-01]
 [ 1.35335283e-01  2.70670566e-01  5.41341133e-01]
 [ 1.11089965e-02  3.33269896e-02  9.99809688e-02]
 [ 3.35462628e-04  1.34185051e-03  5.36740205e-03]
 [ 3.72665317e-06  1.86332659e-05  9.31663293e-05]]
c [1 1 1]
abb[..., j][0] [ 3.72665317e-06 -1.86332659e-05  9.31663293e-05]
G[..., j] [[4.67257133e-310 4.34577426e-114 1.03103233e-259]
 [2.43896053e-154 7.19174982e+140 3.85422801e-110]
 [1.94345452e-109 5.23285868e-067 6.01334434e-154]]
weights [[-1.86332659e-05  9.31663293e-05 -4.65831647e-04]
 [-1.34185051e-03  5.36740205e-03 -2.14696082e-02]
 [-3.33269896e-02  9.99809688e-02 -2.99

RuntimeError: no filter weights given

In [38]:
import skimage.io
img = skimage.io.imread("http://www.hpca.ual.es/~vruiz/images/barb.png")
c = np.ones_like(img)

In [42]:
def poly_exp(f, c, sigma):
    """
    Calculates the local polynomial expansion of a 2D signal, as described by Farneback
    Uses separable normalized correlation
    $f ~ x^T A x + B^T x + C$
    If f[i, j] and c[i, j] are the signal value and certainty of pixel (i, j) then
    A[i, j] is a 2x2 array representing the quadratic term of the polynomial, B[i, j]
    is a 2-element array representing the linear term, and C[i, j] is a scalar
    representing the constant term.
    Parameters
    ----------
    f
        Input signal
    c
        Certainty of signal
    sigma
        Standard deviation of applicability Gaussian kernel
    Returns
    -------
    A
        Quadratic term of polynomial expansion
    B
        Linear term of polynomial expansion
    C
        Constant term of polynomial expansion
    """
    # Calculate applicability kernel (1D because it is separable)
    n = int(4 * sigma + 1)
    x = np.arange(-n, n + 1, dtype=np.int32)
    a = np.exp(-(x**2) / (2 * sigma**2))  # a: applicability kernel [n]

    # b: calculate b from the paper. Calculate separately for X and Y dimensions
    # [n, 6]
    bx = np.stack(
        [np.ones(a.shape), x, np.ones(a.shape), x**2, np.ones(a.shape), x], axis=-1
    )
    by = np.stack(
        [
            np.ones(a.shape),
            np.ones(a.shape),
            x,
            np.ones(a.shape),
            x**2,
            x,
        ],
        axis=-1,
    )

    # Pre-calculate product of certainty and signal
    cf = c * f

    # G and v are used to calculate "r" from the paper: v = G*r
    # r is the parametrization of the 2nd order polynomial for f
    G = np.empty(list(f.shape) + [bx.shape[-1]] * 2)
    v = np.empty(list(f.shape) + [bx.shape[-1]])
    print("G.shape", G.shape)

    # Apply separable cross-correlations

    # Pre-calculate quantities recommended in paper
    ab = np.einsum("i,ij->ij", a, bx)
    abb = np.einsum("ij,ik->ijk", ab, bx)
    #print("a", a, a.shape)
    #print("bx", bx, bx.shape)
    #print("ab", ab)
    #print("abb", abb)
    
    # Calculate G and v for each pixel with cross-correlation
    print("bx.shape[-1]", bx.shape[-1])
    for i in range(bx.shape[-1]):
        for j in range(bx.shape[-1]):
            print("G[..., i, j].shape", G[..., i, j].shape)
            G[..., i, j] = scipy.ndimage.correlate1d(
                c, abb[..., i, j], axis=0, mode="constant", cval=0
            )

        v[..., i] = scipy.ndimage.correlate1d(
            cf, ab[..., i], axis=0, mode="constant", cval=0
        )

    # Pre-calculate quantities recommended in paper
    ab = np.einsum("i,ij->ij", a, by)
    abb = np.einsum("ij,ik->ijk", ab, by)

    # Calculate G and v for each pixel with cross-correlation
    for i in range(bx.shape[-1]):
        for j in range(bx.shape[-1]):
            G[..., i, j] = scipy.ndimage.correlate1d(
                G[..., i, j], abb[..., i, j], axis=1, mode="constant", cval=0
            )

        v[..., i] = scipy.ndimage.correlate1d(
            v[..., i], ab[..., i], axis=1, mode="constant", cval=0
        )

    # Solve r for each pixel
    r = np.linalg.solve(G, v)

    # Quadratic term
    A = np.empty(list(f.shape) + [2, 2])
    A[..., 0, 0] = r[..., 3]
    A[..., 0, 1] = r[..., 5] / 2
    A[..., 1, 0] = A[..., 0, 1]
    A[..., 1, 1] = r[..., 4]

    # Linear term
    B = np.empty(list(f.shape) + [2])
    B[..., 0] = r[..., 1]
    B[..., 1] = r[..., 2]

    # constant term
    C = r[..., 0]

    # b: [n, n, 6]
    # r: [f, f, 6]
    # f: [f, f]
    # e = b*r - f

    return A, B, C

In [43]:
poly_exp(img, c, sigma=1.0)

G.shape (512, 512, 6, 6)
bx.shape[-1] 6
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)
G[..., i, j].shape (512, 512)


(array([[[[ 9.39560674e+01, -2.46357142e+00],
          [-2.46357142e+00, -9.91266278e+01]],
 
         [[ 1.30436249e+02, -2.29520074e+00],
          [-2.29520074e+00, -6.71019430e+01]],
 
         [[ 1.51161768e+02, -2.50021438e+00],
          [-2.50021438e+00, -2.71388713e+01]],
 
         ...,
 
         [[ 1.25137908e+02, -1.25808784e+00],
          [-1.25808784e+00, -2.20090720e+00]],
 
         [[ 1.21018912e+02, -2.69961213e+00],
          [-2.69961213e+00, -1.94417847e+00]],
 
         [[ 1.16981380e+02, -3.51622041e+00],
          [-3.51622041e+00, -1.01553826e+00]]],
 
 
        [[[-1.90282974e+01, -1.53908582e-02],
          [-1.53908582e-02,  7.72256144e-01]],
 
         [[-1.90048268e+01, -4.17971007e-02],
          [-4.17971007e-02,  1.77034418e-01]],
 
         [[-1.90091507e+01,  7.30005023e-02],
          [ 7.30005023e-02,  2.60337739e+00]],
 
         ...,
 
         [[ 5.18044179e+01, -4.12462553e-01],
          [-4.12462553e-01,  1.47823742e+01]],
 
         [[ 3.1

In [7]:
5*3.72665317e-06

1.863326585e-05